In [1]:
from os import listdir
from os.path import isfile, join
import string
import warnings
warnings.filterwarnings("ignore")

In [2]:
my_path = "dataset/20_newsgroups"

In [3]:
#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]

In [4]:
#creating a 2D list to store list of all files in different folders

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [5]:
sum(len(files[i]) for i in range(20))

19997

In [6]:
pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))

In [7]:
Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.25)

In [9]:
import re
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = [] # corpus is collection of text(anything)

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Bhalsod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
for i in range(0, 1000): # 19998 are data but upper bound is exlude so we take 1000
    review = re.sub('[^a-zA-Z]', ' ', X_train[i])
    review = review.lower() # convert all captial letteres to lower
    review = review.split() # it split the string into words
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]# stopwords is all the unnecessary words like the,place,is,that it removes. so ml algorithm can coreleate with best and suitable words
    # we use set() because in large dataset like ariticles it hepls us to work fasetr. words('english') because here we use english comments.
    # ps.stem(word) means it steamms the words like convert loved into love.
    review = ' '.join(review) # we join seperate words in whole word but we the to not combine all 3 words we have to put space and then join the words.
    corpus.append(review)

In [11]:
# define Tokenizer with Vocab Size
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(X_train)

Using TensorFlow backend.


In [12]:
X_train = tokenizer.texts_to_matrix(X_train, mode='tfidf')
X_test = tokenizer.texts_to_matrix(X_test, mode='tfidf')

In [13]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
#encoder.fit(train_tags)
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [14]:
# Building ANN
import keras
from keras.models import Sequential # to initialize neural neiwork
from keras.layers import Dense , Dropout # for densily connected neural network

In [15]:
# INittializing the ANN
model = Sequential()

W0930 23:58:54.195729 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [16]:
# Adding the input layer and first hidden layer
model.add(Dense(activation="relu", input_dim=15000, units=512, kernel_initializer="uniform"))
model.add(Dropout(0.2))

W0930 23:58:54.353535 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 23:58:54.358533 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 23:58:54.400513 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0930 23:58:54.421517 15696 deprecation.py:506] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future 

In [17]:
# Adding the second layer
model.add(Dense(units = 512 , activation='relu' , kernel_initializer = "uniform"))

In [18]:
model.add(Dropout(0.2))
# Output layer
model.add(Dense(units = 20 , activation='softmax' , kernel_initializer = "uniform")) # we use sigmoid because there is only one neuron but if we have ore then 2  output neuron then we have to use softmax

In [19]:
# Compile the ANN
model.compile(optimizer='adam' , loss = 'categorical_crossentropy', metrics= ['accuracy']) # compile means we have to apply algorithm like gradient decent to find optimal set of weights in neuron because first we randomly initialize weight and now we have to apply the algorithm to find optimal set of weights

W0930 23:58:54.973331 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0930 23:58:55.024321 15696 deprecation_wrapper.py:119] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [20]:
model.summary()

# fitting the training set
model.fit(X_train, y_train ,batch_size =80, epochs = 20,verbose=1,validation_split=0.1)

W0930 23:58:55.329326 15696 deprecation.py:323] From C:\Users\Aditya Bhalsod\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                10260     
Total params: 7,953,428
Trainable params: 7,953,428
Non-trainable params: 0
_________________________________________________________________
Train on 13497 samples, validate on 1500 samples
Epoch 1/20
13497/13497 [==============================] - 87s 6ms/step - loss: 0.45